In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install opencv-python ultralytics deep-sort-realtime easyocr pandas numpy openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 90.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalle

In [ ]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import os
from collections import defaultdict
import time
from deep_sort_realtime.deepsort_tracker import DeepSort
import torch

# Initialize device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize models
vehicle_model = YOLO('/kaggle/input/vehicle/pytorch/default/1/best.pt').to(device).half()
plate_model = YOLO('/kaggle/input/license/pytorch/default/1/best_lic.pt').to(device).half()

# Initialize DeepSORT with optimized parameters
deepsort = DeepSort(max_age=20, n_init=1, nn_budget=100, max_cosine_distance=0.4) 

# Video input and output paths
video_path = '/kaggle/input/video-test/tteest.mp4'
output_dir = '/kaggle/working/output/'
output_video_path = os.path.join(output_dir, 'output_video7777777.mp4')
debug_dir = os.path.join(output_dir, 'debug_frames/')
os.makedirs(output_dir, exist_ok=True)
os.makedirs(debug_dir, exist_ok=True)

# Excel output
excel_path = os.path.join(output_dir, 'speeding_vehicles7777777.xlsx')
speeding_data = []

# Speed calculation parameters
speed_limit = 60
reference_width_m = 3.7
reference_pixels = 200
kmh_conversion = 3.6
speed_smoothing = 3
speed_calc_interval = 5
min_vehicle_size = 100
max_vehicle_size = 0.5
min_plate_size = 10
padding = 5
min_confidence = 0.6
min_distance_px = 5

# Video capture
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError(f"Cannot open video file: {video_path}")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Perspective transform setup
src_points = np.float32([
    [frame_width * 0.4, frame_height * 0.2],
    [frame_width * 0.6, frame_height * 0.2],
    [frame_width * 0.2, frame_height * 0.8],
    [frame_width * 0.8, frame_height * 0.8]
])
dst_points = np.float32([
    [0, 0],
    [400, 0],
    [0, 600],
    [400, 600]
])
M = cv2.getPerspectiveTransform(src_points, dst_points)

# Pixel-to-meter scale
scale_m_per_px = reference_width_m / reference_pixels

# Video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking and plate data
vehicle_tracks = defaultdict(list)
plate_results = {}
speed_history = defaultdict(list)

# Font settings (bolder)
font = cv2.FONT_HERSHEY_DUPLEX
font_scale = 1.0
font_thickness = 5  
plate_font_scale = 0.7
plate_font_thickness = 4  

def calculate_speed(positions, times, frame_count, vehicle_id):
    if len(positions) < 2:
        return 0
    speeds = []
    for i in range(1, len(positions)):
        dx = positions[i][0] - positions[i-1][0]
        dy = positions[i][1] - positions[i-1][1]
        distance_px = np.sqrt(dx**2 + dy**2)
        if distance_px < min_distance_px:
            continue
        distance_m = distance_px * scale_m_per_px
        time_diff = times[i] - times[i-1]
        if time_diff > 0:
            speed_ms = distance_m / time_diff
            speed_kmh = speed_ms * kmh_conversion
            speeds.append(speed_kmh)
            print(f"Frame {frame_count}, Vehicle {vehicle_id}: distance_px={distance_px:.1f}, distance_m={distance_m:.2f}, speed={speed_kmh:.1f} km/h")
    return np.mean(speeds[-speed_smoothing:]) if speeds else 0

# Process video frames
frame_count = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    current_time = frame_count / fps

    # Vehicle detection
    vehicle_results = vehicle_model(frame, conf=0.7, iou=0.7, verbose=False)  
    vehicles = vehicle_results[0].boxes

    # Prepare detections for DeepSORT
    detections = []
    max_vehicle_px = frame_width * max_vehicle_size
    for vehicle in vehicles:
        x1, y1, x2, y2 = map(int, vehicle.xyxy[0])
        conf = vehicle.conf.item()
        cls = vehicle.cls.item()
        if conf < min_confidence:
            continue
        shrink = 0.05
        w, h = x2 - x1, y2 - y1
        x1 += int(w * shrink)
        y1 += int(h * shrink)
        x2 -= int(w * shrink)
        y2 -= int(h * shrink)
        if x2 <= x1 or y2 <= y1 or (x2 - x1) < min_vehicle_size or (y2 - y1) < min_vehicle_size or (x2 - x1) > max_vehicle_px:
            print(f"Invalid initial vehicle bbox at frame {frame_count}: x1={x1}, y1={y1}, x2={x2}, y2={y2}, conf={conf:.2f}")
            continue
        bbox = [x1, y1, x2-x1, y2-y1]
        detections.append((bbox, conf, cls))

    # Update DeepSORT
    tracks = deepsort.update_tracks(detections, frame=frame)

    # Process tracks
    for track in tracks:
        if not track.is_confirmed():
            continue
        vehicle_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_tlbr())
        conf = track.get_det_conf() or 0.5
        cls = track.get_det_class() or 0

        # Validate and clip vehicle bounding box
        x1 = max(0, min(x1, frame_width - 1))
        x2 = max(0, min(x2, frame_width - 1))
        y1 = max(0, min(y1, frame_height - 1))
        y2 = max(0, min(y2, frame_height - 1))
        if x2 <= x1 or y2 <= y1 or (x2 - x1) < min_vehicle_size or (y2 - y1) < min_vehicle_size or (x2 - x1) > max_vehicle_px:
            print(f"Invalid vehicle bbox at frame {frame_count}: x1={x1}, y1={y1}, x2={x2}, y2={y2}")
            cv2.imwrite(f"{debug_dir}/invalid_vehicle_frame_{frame_count}_{vehicle_id}.jpg", frame)
            continue

        # Calculate center
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2

        # Transform center to bird's-eye view
        point = np.array([[center_x, center_y]], dtype=np.float32)
        point = point.reshape(-1, 1, 2)
        warped_point = cv2.perspectiveTransform(point, M)
        warped_x, warped_y = warped_point[0, 0]

        # Store warped coordinates for speed calculation
        vehicle_tracks[vehicle_id].append(((warped_x, warped_y), current_time))

        # Calculate speed
        smoothed_speed = 0
        if frame_count % speed_calc_interval == 0 or not speed_history[vehicle_id]:
            positions, times = zip(*[(pos, t) for pos, t in vehicle_tracks[vehicle_id]])
            speed = calculate_speed(positions, times, frame_count, vehicle_id)
            speed_history[vehicle_id].append(speed)
            smoothed_speed = np.mean(speed_history[vehicle_id][-speed_smoothing:]) if speed_history[vehicle_id] else 0
        else:
            smoothed_speed = speed_history[vehicle_id][-1] if speed_history[vehicle_id] else 0

        # Draw vehicle box
        color = (0, 255, 0) if smoothed_speed <= speed_limit else (0, 0, 255)  
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 8)  
        label = f"ID: {vehicle_id} Conf: {conf:.2f} Speed: {smoothed_speed:.1f} km/h"
        cv2.putText(frame, label, (x1, y1 - 30), font, font_scale, color, font_thickness)

        # License plate detection
        try:
            crop = frame[y1:y2, x1:x2]
            if crop.shape[0] < min_vehicle_size or crop.shape[1] < min_vehicle_size:
                print(f"Invalid crop size at frame {frame_count}: height={crop.shape[0]}, width={crop.shape[1]}")
                cv2.imwrite(f"{debug_dir}/invalid_crop_frame_{frame_count}_{vehicle_id}.jpg", crop)
                continue
            plate_results_frame = plate_model(crop, conf=0.65, verbose=False)  
            plates = plate_results_frame[0].boxes
        except Exception as e:
            print(f"Plate detection failed at frame {frame_count}: {str(e)}")
            cv2.imwrite(f"{debug_dir}/failed_plate_frame_{frame_count}_{vehicle_id}.jpg", frame)
            continue

        for plate in plates:
            px1, py1, px2, py2 = map(int, plate.xyxy[0])
            pconf = plate.conf.item()

            # Add padding
            px1 = max(0, px1 - padding)
            py1 = max(0, py1 - padding)
            px2 = min(crop.shape[1], px2 + padding)
            py2 = min(crop.shape[0], py2 + padding)

            # Convert to frame coordinates
            px1, py1, px2, py2 = px1+x1, py1+y1, px2+x1, py2+y1

            # Validate plate box
            px1 = max(0, min(px1, frame_width - 1))
            px2 = max(0, min(px2, frame_width - 1))
            py1 = max(0, min(py1, frame_height - 1))
            py2 = max(0, min(py2, frame_height - 1))
            if px2 <= px1 or py2 <= py1 or (px2 - px1) < min_plate_size or (py2 - py1) < min_plate_size:
                print(f"Invalid plate bbox at frame {frame_count}: px1={px1}, py1={py1}, px2={px2}, py2={py2}, conf={pconf:.2f}")
                cv2.imwrite(f"{debug_dir}/invalid_plate_frame_{frame_count}_{vehicle_id}.jpg", frame[py1:py2, px1:px2] if py2 > py1 and px2 > px1 else frame)
                continue

            # Store plate result
            plate_area = (px2 - px1) * (py2 - py1)
            if vehicle_id not in plate_results or (pconf > plate_results[vehicle_id][2] and plate_area > plate_results[vehicle_id][3]):
                plate_results[vehicle_id] = (frame_count, pconf, plate_area, (px1, py1, px2, py2))

            # Draw plate box
            cv2.rectangle(frame, (px1, py1), (px2, py2), (255, 0, 0), 4) 
            plate_label = f"Plate Conf: {pconf:.2f}"
            cv2.putText(frame, plate_label, (px1, py1 - 20), font, plate_font_scale, (255, 0, 0), plate_font_thickness)

            # Save to Excel if speeding
            if smoothed_speed > speed_limit:
                speeding_data.append({
                    'Vehicle_ID': vehicle_id,
                    'Speed_kmh': smoothed_speed,
                    'Frame': frame_count,
                    'Timestamp': current_time,
                    'Vehicle_Class': cls,
                    'Plate_BBox': f"({px1}, {py1}, {px2}, {py2})"
                })

    # Write frame
    out.write(frame)

    # Progress
    if frame_count % 100 == 0:
        print(f"Processed {frame_count}/{total_frames} frames")

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Save Excel
if speeding_data:
    df = pd.DataFrame(speeding_data)
    df.to_excel(excel_path, index=False)
    print(f"Speeding data saved to {excel_path}")
else:
    print("No speeding vehicles detected.")

print(f"Output video saved at {output_video_path}")